# 데이터 수집

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import html

In [ ]:
# naver api key 설정
with open("./drive/MyDrive/실습/RAG/api_key.json", 'r') as j :
    json_key = json.load(j)


In [ ]:
client_id = json_key['client_id']
client_secret = json_key['client_secret']

In [ ]:
def getRequestUrl(url) :
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try :
        response = urllib.request.urlopen(req)
        if response.getcode() == 200 :
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e :
        print(e)
        print("[%s] Error for URL : %s" %(datetime.datetime.now(), url))
        return None

def getNaverSearch(node, srcText, start, display) :
    base = "https://openapi.naver.com/v1/search/"
    node = "%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if (responseDecode == None) :
        return None
    else :
        return json.loads(responseDecode)

def clean_text(text):
    # 유니코드 이스케이프 → 실제 문자로
    # text = text.encode('utf-8').decode('unicode_escape')
    # HTML 엔티티 디코드 (&amp; 등)
    text = html.unescape(text)
    # 간단한 HTML 태그 제거 (<b> 등)
    import re
    text = re.sub(r'<[^>]+>', '', text)

    return text

def getPostData(post, jsonResult, cnt) :
    title = clean_text(post['title'])
    description = clean_text(post['description'])
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt' : cnt, 'title': title, 'description' : description,
                       'org_link' : org_link, 'link' : org_link, 'pDate' : pDate})

    return

In [ ]:
def getNews(src) :
    node = 'news' # 크롤링 대상
    srcText = src
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    print(jsonResponse)
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)) :
        for post in jsonResponse['items'] :
            cnt += 1
            getPostData(post, jsonResult, cnt)

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    print("전체 검색 : %d 건" %total)

    with open('./drive/MyDrive/실습/RAG/data/%s_naver_%s.json' %(srcText, node), 'w', encoding='utf-8') as output :
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

        output.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))
    print("%s_naver_%s.json SAVED" %(srcText, node))

## 뉴스 크롤링

In [ ]:
getNews('lck')

[2025-06-02 06:00:09.909295] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:00:09 +0900', 'total': 123728, 'start': 1, 'display': 100, 'items': [{'title': '[<b>LCK</b>] 25시즌 2라 종료... 무적의 젠지, DK-KT는 순위 결정전으로', 'originallink': 'https://www.fomos.kr/redirect/news_view?news_cate_id=39&entry_id=144791', 'link': 'https://m.sports.naver.com/esports/article/236/0000249301', 'description': "2025 <b>LCK</b> 정규 시즌 1라운드를 9전 전승으로 마무리했던 젠지가 2라운드에서도 패배를 당하지 않으면서 18전... 라이엇 게임즈가 개발 및 서비스하는 '리그 오브 레전드(LoL)'의 이스포츠 한국 프로 리그인 <b>LCK</b>를... ", 'pubDate': 'Mon, 02 Jun 2025 14:59:00 +0900'}, {'title': '2025 <b>LCK</b> CL 정규시즌 2라운드 종료, 챌린지 그룹 5팀 최종 확정', 'originallink': 'https://www.fomos.kr/redirect/news_view?news_cate_id=39&entry_id=144789', 'link': 'https://m.sports.naver.com/esports/article/236/0000249299', 'description': '2025 <b>LCK</b> 챌린저스 리그 정규시즌 2라운드가 30일(금) 경기를 끝으로 마무리되며, 챌린지 그룹과 트라이얼... <b>LCK</b> CL은 약 한달 반가량의 휴식기를 가진 뒤, 오는 7월 21일(월) 3라운드로 돌아온다. 3라운드는 챌린지 그룹과... ', 'pubDate': 'Mon, 02

In [ ]:
getNews('대선')

[2025-06-02 06:00:20.985986] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:00:20 +0900', 'total': 4473805, 'start': 1, 'display': 100, 'items': [{'title': '부자감세로 곳간 비었는데 또 감세… 무슨 돈으로 공약 지킬 겁니까? [공...', 'originallink': 'https://www.thescoop.co.kr/news/articleView.html?idxno=306034', 'link': 'https://n.news.naver.com/mnews/article/665/0000005128?sid=101', 'description': "&quot; # 6ㆍ3 <b>대선</b>에 출사표를 던졌던 후보 중 어떤 이는 '국채 발행'을, 또 다른 어떤 이는 '증세'를 방법론으로... ☞ 참고: 6ㆍ3 <b>대선</b> 에디션 '공약논쟁前'의 취지는 공약을 논쟁하기 전前에 논쟁해야 할 이슈를 살펴보자는... ", 'pubDate': 'Mon, 02 Jun 2025 14:59:00 +0900'}, {'title': "선거 막판 '댓글부대' 논란 &quot;공작 확실..국힘과 연관성 핵심&quot;..&quot;자발적 ...", 'originallink': 'https://www.ikbc.co.kr/article/view/kbc202506020033', 'link': 'https://n.news.naver.com/mnews/article/660/0000086411?sid=154', 'description': "<b>대선</b>을 앞두고 불거진 이른바 '리박스쿨' 댓글조작 의혹과 관련해 &quot;댓글 공작은 확실한데 국민의힘과... 그걸 할 자격도 지금 이재명 후보는 전혀 없는 상태&quot;라고 말했습니다. #시사1번지 #리박스쿨 #자손군 #손가혁 #장미<b>대선</b>", 'pubDate': 'Mon, 02 Jun 2025 

In [ ]:
getNews('중국')

[2025-06-02 06:00:31.157890] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:00:31 +0900', 'total': 13781218, 'start': 1, 'display': 100, 'items': [{'title': "한화에어로스페이스, 韓기업 최초 '샹그릴라 대화' 참석…국방비 증액 ...", 'originallink': 'http://www.wikileaks-kr.org/news/articleView.html?idxno=169281', 'link': 'http://www.wikileaks-kr.org/news/articleView.html?idxno=169281', 'description': "이번 샹그릴라 대화 연설에서 피트 헤그세스 미국 국방부 장관은 지난 31일(현지시간) <b>중국</b>의 대만 침공 가능성을 경고, 동맹국들을 향해 '<b>중국</b>과의 경제적 협력을 경계하라'고 했다. 헤그세스 장관은 &quot;동맹과... ", 'pubDate': 'Mon, 02 Jun 2025 14:58:00 +0900'}, {'title': 'EU vs 미국 vs <b>중국</b>…누가 AI의 미래를 좌우할까', 'originallink': 'https://zdnet.co.kr/view/?no=20250602145742', 'link': 'https://n.news.naver.com/mnews/article/092/0002376735?sid=105', 'description': "데이터법, <b>중국</b> 생성형 AI 서비스 임시 조치, 브라질 AI 법안 2338/2023 등 5개 주요 법안을 분석했다.... 디지털 선진국 vs 후발주자: EU·<b>중국</b>은 성숙, 미국은 '연방법 공백' 상태 AI 규제는 각국의 디지털 법적 환경... ", 'pubDate': 'Mon, 02 Jun 2025 14:58:00 +0900'}, {'title': '&quot;자연 속 힐링

In [ ]:
getNews('살해')

[2025-06-02 06:00:39.328151] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:00:39 +0900', 'total': 1092661, 'start': 1, 'display': 100, 'items': [{'title': '[티조 Clip] 2030 여성들, 이재명 아들 막말에 폭발 &quot;여성 존엄 짓밟힌 ...', 'originallink': 'https://news.tvchosun.com/site/data/html_dir/2025/06/02/2025060290158.html', 'link': 'https://n.news.naver.com/mnews/article/448/0000531972?sid=154', 'description': '2030 여성들 &quot;이재명은 여자친구 <b>살해</b>한 조카 감형 요구&quot;', 'pubDate': 'Mon, 02 Jun 2025 14:53:00 +0900'}, {'title': '“아버지 꾸짖음에 분노”… 호치민시 아버지 살인에 한국인 사형 결정', 'originallink': 'https://lawtalknews.co.kr/article/LNZZH6FXDWFF', 'link': 'https://lawtalknews.co.kr/article/LNZZH6FXDWFF', 'description': '베트남 호치민시에서 친아버지를 흉기로 <b>살해</b>한 한국인 남성에게 현지 법원이 사형을 선고했다.... 지난달 29일(현지시각) 베트남 매체 타이니엔과 뚜오이쩨 등에 따르면, 호치민시 인민법원은 아버지 <b>살해</b> 혐의로 기소된... ', 'pubDate': 'Mon, 02 Jun 2025 14:48:00 +0900'}, {'title': '20대 한인 남성, 5세된 딸 무자비 폭행 <b>살해</b>', 'originallink': 'http://www.koreatimes.com/article/1566690', 'link'

In [ ]:
getNews('메달')

[2025-06-02 06:00:47.350825] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:00:47 +0900', 'total': 656438, 'start': 1, 'display': 100, 'items': [{'title': '넷마블조정선수단, 전국대회서 <b>메달</b> 10개 수확', 'originallink': 'http://www.sportsq.co.kr/news/articleView.html?idxno=481242', 'link': 'http://www.sportsq.co.kr/news/articleView.html?idxno=481242', 'description': "자릿수 <b>메달</b>을 수확했다. 넷마블문화재단은 &quot;'넷마블조정선수단'이 지난달 30일과 31일, 양일간 경기도 하남시 미사경정공원 조정경기장에서 열린 '2025 전국장애인조정선수권대회'에서 총 10개(금3·은2·동5)의 <b>메달</b>을... ", 'pubDate': 'Mon, 02 Jun 2025 14:56:00 +0900'}, {'title': '섬뜩한 아우라 오정세, 新 빌런 탄생 예고', 'originallink': 'https://www.sportsseoul.com/news/read/1520276?ref=naver', 'link': 'https://m.entertain.naver.com/article/468/0001151540', 'description': 'JTBC 새 토일드라마 ‘굿보이’(극본 이대일, 연출 심나연, 제작 SLL·스튜디오앤뉴·드라마하우스스튜디오)는 특채로 경찰이 된 <b>메달</b>리스트들이 <b>메달</b> 대신 경찰 신분증을 목에 걸고, 비양심과 반칙이 판치는 세상에 맞서... ', 'pubDate': 'Mon, 02 Jun 2025 14:51:00 +0900'}, {'title': '넷마블조정선수단, 전국장애인조정선수권대회서 총 10개 <b>메달</b> 획득', 

In [ ]:
getNews('금리')

[2025-06-02 06:00:55.997612] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:00:55 +0900', 'total': 4047403, 'start': 1, 'display': 100, 'items': [{'title': '트럼프 영국과 첫 탈관세 무역협정 체결 확정에 비트코인 10만 달러 돌파...', 'originallink': 'https://www.tokenpost.kr/news/cryptocurrency/245991', 'link': 'https://www.tokenpost.kr/news/cryptocurrency/245991', 'description': '&quot;시장이 <b>금리</b>보다 훨씬 더 신경 쓰는 것은 관세에 관한 논의이며, 트럼프 대통령은 방금 위험 자산에 큰 생명줄을 던졌다.&quot; 그러나 퍼크린은 주의를 당부했다. &quot;발표에 구체적인 세부 사항이 부족할 가능성이 높아... ', 'pubDate': 'Mon, 02 Jun 2025 14:58:00 +0900'}, {'title': '하동군, 다층적 청년 주거정책으로 장기 정착 유도', 'originallink': 'https://www.newsfreezone.co.kr/news/articleView.html?idxno=629667', 'link': 'https://www.newsfreezone.co.kr/news/articleView.html?idxno=629667', 'description': '◇현실적 부담 완화에서 시작된 정책…높은 체감도 입증 <b>금리</b>·고물가가 지속되는 상황 속에서 청년층의 주거 부담은 더욱 가중되고 있다. 하동군이 시행한 청년정책 수요 조사에서도 주거 분야는 가장 높은 관심과... ', 'pubDate': 'Mon, 02 Jun 2025 14:58:00 +0900'}, {'title': '리플(XRP), FOMC 회의 직후 3.3% 상승하며 2.21달러 기록

In [ ]:
getNews('더위')

[2025-06-02 06:01:04.041368] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:01:03 +0900', 'total': 1387444, 'start': 1, 'display': 100, 'items': [{'title': "역대급 <b>더위</b> 예고…에이블리·지그재그, 폭염 대비템 인기 '쑥'", 'originallink': 'https://daily.hankooki.com/news/articleView.html?idxno=1224452', 'link': 'https://daily.hankooki.com/news/articleView.html?idxno=1224452', 'description': "사진=에이블리 제공 서울의 한낮 기온이 최대 30도까지 오르는 등 올해 역대급 무<b>더위</b>가 예고되면서 에이블리, 지그재그 등 MZ세대 주요 패션 플랫폼들의 '여름 대비템' 거래액이 큰 폭으로 올랐다. 무<b>더위</b>와 높은... ", 'pubDate': 'Mon, 02 Jun 2025 14:58:00 +0900'}, {'title': '[오늘의 여가] 쏠비치 남해·롯데호텔 제주·켄싱턴리조트·호시노 리조...', 'originallink': 'http://www.financialreview.co.kr/news/articleView.html?idxno=34971', 'link': 'http://www.financialreview.co.kr/news/articleView.html?idxno=34971', 'description': '제주는 일찍 무<b>더위</b>가 시작돼 수영장 개장전부터 고객 문의가 이어져 현재 개장일부터 일주일간 평균 90% 예약률을 기록하고 있다. 야외 수영장은 9월 14일(일)까지 매일 오전 9시부터 오후 6시까지 운영되며, 성수기 기간... ', 'pubDate': 'Mon, 02 Jun 2025 14:56:00 +0900'}, {'title': '파

In [ ]:
getNews('연휴')

[2025-06-02 06:01:11.778405] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:01:11 +0900', 'total': 2061239, 'start': 1, 'display': 100, 'items': [{'title': '[KINN] ‘한덕수 대망론’, 언론개혁 절실함을 보여주다', 'originallink': 'https://newstapa.org/article/_drwk', 'link': 'https://n.news.naver.com/mnews/article/607/0000002724?sid=102', 'description': '의료 대란에 대해 기자가 “큰 사고 없이 (추석)<b>연휴</b>가 지나갔다” 라고 하자, 한 총리는 의료 대란을 전공의와 과거 정부 탓으로 돌렸다. 이른바 ‘응급실 뺑뺑이’로 사망자도 나왔지만 언급하지 않았다. 한덕수 총리는... ', 'pubDate': 'Mon, 02 Jun 2025 15:00:00 +0900'}, {'title': "대전 국내 여행지 점유율 증가율'전국 1위'기록", 'originallink': 'http://www.bzeronews.com/news/articleView.html?idxno=713376', 'link': 'http://www.bzeronews.com/news/articleView.html?idxno=713376', 'description': "온라인 여행기업 '놀유니버스'가 발표한 황금<b>연휴</b>(5월 1~6일) 기간 숙박 예약 현황에서도 대전은 예약 건수가 전년 대비 무려 190% 증가하며 전국 1위를 차지했다. 또한 한국관광공사의 2024년 지역별 방문객 통계에 따르면... ", 'pubDate': 'Mon, 02 Jun 2025 14:58:00 +0900'}, {'title': "대전시, 국내 여행지 점유율 증가율 '전국 1위' 기록", 'originallink': 'https://www.newscj.

In [ ]:
getNews('과일')

[2025-06-02 06:01:20.753469] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:01:20 +0900', 'total': 1139570, 'start': 1, 'display': 100, 'items': [{'title': "부탄, 관광객 암호화폐 결제 허용...'세계 최초 국가 수준 암호화폐 관...", 'originallink': 'https://www.tokenpost.kr/news/blockchain/245554', 'link': 'https://www.tokenpost.kr/news/blockchain/245554', 'description': '이 새로운 시스템은 바이낸스 계정을 가진 여행자들이 항공권과 비자 수수료부터 호텔 숙박, 투어, 심지어 길가의 <b>과일</b> 노점까지 다양한 결제에 디지털 자산을 사용할 수 있게 한다. 디케이뱅크와 바이낸스페이에 의해... ', 'pubDate': 'Mon, 02 Jun 2025 15:00:00 +0900'}, {'title': "롯데웰푸드, 프랑스 대표 유제품 기업 락탈리스社와 함께 '글로벌 셰프...", 'originallink': 'https://www.ibabynews.com/news/articleView.html?idxno=132962', 'link': 'https://www.ibabynews.com/news/articleView.html?idxno=132962', 'description': "산뜻한 <b>과일</b> 풍미가 어우러진 '오리엔탈 타르틀렛'을 선보였다. 글로벌 무대에서 쌓은 경험을 바탕으로 학생들에게 베이킹 기술을 가르치고 있는 제레미 볼레스터 셰프와 니콜라 피에로 셰프의 시연 소식은 시작 전부터... ", 'pubDate': 'Mon, 02 Jun 2025 14:54:00 +0900'}, {'title': '포스코 광양제철소 에너지부, 골약동 용장마을과 25년째 아름다운 동행', 'originallink'

In [ ]:
getNews('축구')

[2025-06-02 06:01:29.143715] Url Request Success
{'lastBuildDate': 'Mon, 02 Jun 2025 15:01:29 +0900', 'total': 5096001, 'start': 1, 'display': 100, 'items': [{'title': "홍명보호 11회 연속 월드컵 진출 확정하러 이라크로…손흥민·이강인 '우...", 'originallink': 'https://www.mediapen.com/news/view/1019807', 'link': 'https://www.mediapen.com/news/view/1019807', 'description': '한국 <b>축구</b>대표팀 홍명보호가 11회 연속 월드컵 본선 진출을 확정하러 이라크로 향했다. 홍명보 감독이 이끄는 한국 <b>축구</b>대표팀은 2일 인천국제공항을 통해 전세기편을 이용, 이라크로 출국했다. 한국은 오는 6일 오전 3시... ', 'pubDate': 'Mon, 02 Jun 2025 15:00:00 +0900'}, {'title': '쿠팡플레이, 홍명보호 월드컵 예선 이라크·쿠웨이트전 생중계', 'originallink': 'http://www.stoo.com/article.php?aid=100740597638', 'link': 'http://www.stoo.com/article.php?aid=100740597638', 'description': "쿠팡플레이가 대한민국 <b>축구</b> 대표팀의 '11회 연속 월드컵 본선 진출'이라는 대기록이 걸린 운명의 순간을... 이번 2연전은 <b>축구</b> 팬들에게 유럽파 스타들을 한자리에서 만날 수 있는 특별한 기회다. 토트넘 홋스퍼의... ", 'pubDate': 'Mon, 02 Jun 2025 15:00:00 +0900'}, {'title': "레노버-모토로라, '2025 FIFA 클럽 월드컵' 공식 파트너십 체결", 'originallink': 'https://cwn.kr/articl

# LangChain

In [ ]:
!pip install -qU "langchain[openai]"
!pip install -qU langchain-openai
!pip install -qU langchain-core
!pip install -qU langgraph
!pip install -qU langchain_community
!pip install -qU jq
!pip install -qU langchain_chroma
!pip install -qU langchain_experimental
!pip install -qU langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... d

In [ ]:
import os
import getpass

# openai api key 설정
with open("./drive/MyDrive/실습/RAG/openai_api_key.json", 'r') as j :
    openai_key = json.load(j)

os.environ["OPENAI_API_KEY"] = openai_key['OPENAI_API_KEY']


In [ ]:
data_path = './drive/MyDrive/실습/RAG/data/'
json_paths = [data_path + json_file for json_file in os.listdir(data_path)]

In [ ]:
json_paths

['./drive/MyDrive/실습/RAG/data/lck_naver_news.json',
 './drive/MyDrive/실습/RAG/data/대선_naver_news.json',
 './drive/MyDrive/실습/RAG/data/중국_naver_news.json',
 './drive/MyDrive/실습/RAG/data/살해_naver_news.json',
 './drive/MyDrive/실습/RAG/data/메달_naver_news.json',
 './drive/MyDrive/실습/RAG/data/금리_naver_news.json',
 './drive/MyDrive/실습/RAG/data/더위_naver_news.json',
 './drive/MyDrive/실습/RAG/data/연휴_naver_news.json',
 './drive/MyDrive/실습/RAG/data/과일_naver_news.json',
 './drive/MyDrive/실습/RAG/data/축구_naver_news.json']

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import JSONLoader
from langchain_chroma import Chroma

# embedding 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# ChromaDB Path
DB_PATH = "./drive/MyDrive/실습/RAG/db"

# Json 파일 Path
data_path = './drive/MyDrive/실습/RAG/data/'
json_paths = [data_path + json_file for json_file in os.listdir(data_path)]

# Document 로드 후 DB에 저장
for i, json_path in enumerate(json_paths) :
    loader = JSONLoader(
        file_path=json_path,
        jq_schema=".[] | .link + \" \" + .pDate + \" \" + .title + \" \" + .description",
        text_content=False
    )

    docs = loader.load()

    if i == 0 :
        db = Chroma.from_documents(
            documents=docs,
            embedding=embeddings,
            collection_name="2025_news",
            persist_directory=DB_PATH
        )

        db.get()

    else :
        db.add_documents(
            documents=docs,
            embedding=embeddings,
            collection_name="2025_news",
            persist_directory=DB_PATH
        )
        db.get()



In [ ]:
db.get()

{'ids': ['6fff5ffd-d6ea-4625-88cd-b91a6ce2acac',
  'e045f2fe-3b17-4c5a-80cf-6a8e81e46abf',
  'aaead529-a1dd-4bba-be8e-055e2628f2e1',
  '89539708-21dc-4680-a9be-2f0f63212f59',
  '73c86a7f-136b-4687-901b-2115a10a1329',
  'bce71053-a9e2-4849-a883-d03215fe1754',
  '7e844b80-9abd-43a6-8979-fef7b02c2b30',
  'c6443e0e-b51b-4f02-93c0-6bccbae860a1',
  'cef99f99-f22c-4013-85c8-c35a5ce0680e',
  '99555cfb-4af4-44cf-96cb-9e061d31dfa7',
  '325b4f9d-1424-4f58-aec1-c247f2f583eb',
  '833fa3dc-9ad8-4200-b243-d2c3a47e6dc6',
  '85c013a0-8454-4ffa-a939-6a2d98fdfe93',
  '8c51825f-1c18-4469-a2dc-2fade43ec634',
  '15d1f2e5-95a8-49a6-863d-86acc73392e6',
  '16fb5ed7-419a-40a2-90f1-8615493bd7c5',
  'c074c24d-32e7-42bb-bec4-dede429b9752',
  'd1601e7b-4983-4679-85a2-3c968211be0f',
  '13cecef5-c089-4525-936b-9572ba6d07f2',
  '45e08b01-dbda-4712-b419-8a1db2547f29',
  '745d1983-d689-4032-8b1b-0d585db2189f',
  '02740cf8-d991-4b3c-92f8-011b45a120b0',
  '75c7ec2b-fde9-4168-8d9f-c168bd6c1f2e',
  'ac9118ca-71d8-447d-8b6c-

In [ ]:
# Retriever
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="2025_news"
)

retriever = persist_db.as_retriever()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

llm = init_chat_model('gpt-4o-mini', model_provider='openai')

def format_docs(docs) :
    return "\n\n".join(doc.page_content for doc in docs)

template = """
Question-Answering task Assistant 역할을 맡아주세요. retrieved context를 사용하여 질문에 답변해주세요.
만약에 답을 알지 못한다면, 모른다고 답해주세요.
최대 3문장으로 간결하게 답해주시고, 답변에 사용한 document를 인용해주세요(url과 날짜).

질문 : {question}

Context : {context}

Answer :
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "오늘은 2025년 6월 4일입니다. 최근 한화와 T1의 전적이 어떻게 되나요?"
rag_chain.invoke(question)

'최근 한화생명e스포츠는 T1을 상대로 2:0으로 승리하여 2위 자리를 지켰습니다. 이 결과로 한화생명은 T1에 대한 천적 관계를 유지하게 되었습니다. (출처: https://www.inven.co.kr/webzine/news/?news=306341, 2025-05-28)'

In [ ]:
question1 = "오늘은 2025 6월 4일입니다. 최근 살인 사건과 관련한 뉴스는 무엇이 있나요?"
rag_chain.invoke(question1)

'최근 발생한 사건 중 하나는 시흥시에서의 연쇄 살인 사건입니다. 피의자 A 씨는 형과 동생을 둔기로 공격해 각각 살해한 것으로 보이며, 범행 후에는 일상적인 행동을 이어갔다고 합니다. (출처: https://www.newsfreezone.co.kr/news/articleView.html?idxno=628543, 2025-05-27)'

# 참고
[NAVER API 사용하여 크롤링](https://wingyu-story.tistory.com/4)

[LangChain RAG 실습](https://day-to-day.tistory.com/76)

[jq schema](https://creboring.net/blog/how-to-use-jq/)

[Langchain API Document](https://python.langchain.com/api_reference)